In [12]:
from jet.llm.ollama.constants import OLLAMA_LARGE_EMBED_MODEL, OLLAMA_SMALL_LLM_MODEL
from jet.llm.ollama.models import OLLAMA_MODEL_EMBEDDING_TOKENS


data_dir = "/Users/jethroestrada/Desktop/External_Projects/Jet_Projects/JetScripts/data/jet-resume/data"
store_path = "/Users/jethroestrada/Desktop/External_Projects/Jet_Projects/JetScripts/test/graphs/generated/run_deeplake/pg_essay_deeplake"

llm_model = OLLAMA_SMALL_LLM_MODEL
embed_model = OLLAMA_LARGE_EMBED_MODEL
chunk_size: int = OLLAMA_MODEL_EMBEDDING_TOKENS[embed_model]
chunk_overlap: int = 40

In [13]:
from jet.llm.ollama.base import initialize_ollama_settings
initialize_ollama_settings({
    "llm_model": llm_model,
    "embed_model": embed_model,
})

_EnhancedSettings(_llm=Ollama(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x3237e7b00>, system_prompt=None, messages_to_prompt=<function messages_to_prompt at 0x322391ee0>, completion_to_prompt=<function default_completion_to_prompt at 0x322460680>, output_parser=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>, query_wrapper_prompt=None, base_url='http://localhost:11434', model='llama3.2', temperature=0.1, context_window=3900, request_timeout=300.0, prompt_key='prompt', json_mode=False, additional_kwargs={}, is_function_calling_model=True, keep_alive=None, max_tokens=0.4), _embed_model=OllamaEmbedding(model_name='nomic-embed-text', embed_batch_size=32, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x3237e7b00>, num_workers=None, base_url='http://localhost:11434', ollama_additional_kwargs={}), _callback_manager=None, _tokenizer=None, _node_parser=SentenceSplitter(include_metadata=True, include_prev_next_r

In [14]:
from jet.llm.ollama.base import OllamaEmbedding


def ollama_embedding_function(texts, model=embed_model):
    if isinstance(texts, str):
        texts = [texts]

    embed_model = OllamaEmbedding(model_name=model)
    results = embed_model.get_general_text_embedding(texts)
    return results


embedding_function = ollama_embedding_function

In [15]:
from jet._token.token_utils import get_ollama_tokenizer
from llama_index.core.node_parser.text.sentence import SentenceSplitter
from llama_index.core.readers.file.base import SimpleDirectoryReader


documents = SimpleDirectoryReader(data_dir).load_data()

In [16]:
len(documents)

7

In [17]:
splitter = SentenceSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    tokenizer=get_ollama_tokenizer(embed_model).encode
)
all_nodes = splitter.get_nodes_from_documents(documents, show_progress=True)

Parsing nodes:   0%|          | 0/7 [00:00<?, ?it/s]

In [18]:
len(all_nodes)

7

In [19]:
import deeplake
ds = deeplake.load(store_path)

/Users/jethroestrada/Desktop/External_Projects/Jet_Projects/JetScripts/test/graphs/generated/run_deeplake/pg_essay_deeplake loaded successfully.



In [20]:
ds.summary()

Dataset(path='/Users/jethroestrada/Desktop/External_Projects/Jet_Projects/JetScripts/test/graphs/generated/run_deeplake/pg_essay_deeplake', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
 embedding  embedding  (14, 768)  float32   None   
    id        text      (14, 1)     str     None   
 metadata     json      (14, 1)     str     None   
   text       text      (14, 1)     str     None   


In [21]:
from deeplake.core.vectorstore import VectorStore
from jet.transformers.formatters import format_json
from jet.logger import logger

# Perform a vector search
vector_store = VectorStore(path=store_path, embedding_function=embedding_function)

Deep Lake Dataset in /Users/jethroestrada/Desktop/External_Projects/Jet_Projects/JetScripts/test/graphs/generated/run_deeplake/pg_essay_deeplake already exists, loading from the storage


In [34]:
query = "Tell me about yourself and your recent achievements."
results = vector_store.search(
    embedding_data=query,
    k=4,
)
logger.info(query)
logger.success(format_json(results))

Calling OllamaEmbedding embed...
Embed model: nomic-embed-text (768)
Batch Tokens: 768
Tell me about yourself and your recent achievements.
{
  "score": [
    0.5524558424949646,
    0.5524558424949646,
    0.5063320994377136,
    0.5063320994377136
  ],
  "id": [
    "8bc6f894-d73e-11ef-98f0-32e9cbb94093",
    "5e24e43c-d73e-11ef-aab6-32e9cbb94093",
    "8bc6f718-d73e-11ef-98f0-32e9cbb94093",
    "5e24e22a-d73e-11ef-aab6-32e9cbb94093"
  ],
  "metadata": [
    "project_mobile_apps.md",
    "project_mobile_apps.md",
    "companies.md",
    "companies.md"
  ],
  "text": [
    "# Mobile apps\n\na.) JABA AI\nAchievements:\n- Worked under pressure to but delivered initial version of web app for investor demo\n- Developed iOS app from scratch and deployed multiple App Store release versions\n- Key features: Chatbot, TikTok - Instagram - Youtube video responses\n- Technologies used: React, React Native, Expo\n\nb.) Graduapp\nAchievements:\n- As one of the pioneers, I was involved in the selec

In [35]:
query = "Provide your primary and secondary skills."
results = vector_store.search(
    embedding_data=query,
    k=4,
)
logger.info(query)
logger.success(format_json(results))

Calling OllamaEmbedding embed...
Embed model: nomic-embed-text (768)
Batch Tokens: 768
Provide your primary and secondary skills.
{
  "score": [
    0.5916950702667236,
    0.5916950702667236,
    0.4691757261753082,
    0.4691757261753082
  ],
  "id": [
    "8bc6f948-d73e-11ef-98f0-32e9cbb94093",
    "5e24e4f0-d73e-11ef-aab6-32e9cbb94093",
    "8bc6f90c-d73e-11ef-98f0-32e9cbb94093",
    "5e24e4be-d73e-11ef-aab6-32e9cbb94093"
  ],
  "metadata": [
    "skills.md",
    "skills.md",
    "recent.md",
    "recent.md"
  ],
  "text": [
    "# Skills\n\nFrontend:\n- React\n- React Native\n- Vanilla JS/CSS\n- Expo\n- GraphQL\n- Redux\n- Gatsby\n- TypeScript\nBackend:\n- Node.js\n- Python\nDatabases:\n- PostgreSQL\n- MongoDB\nPlatforms:\n- Firebase\n- AWS\n- Google Cloud\nDeveloper Tools:\n- Photoshop\n- Jest (Unit testing)\n- Cypress (Integration testing)\n- Selenium (E2E testing)\n- Git\n- Sentry bug tracker\n- Android Studio\n- Xcode\n- Fastlane\n- Serverless\n- ChatGPT",
    "# Skills\n\nFro

In [36]:
ds.visualize()